# Calculation of the photon survival probability

## NGC1275 in the center of the Perseus galaxy cluster

Having in mind that the NGC1275 is located in a rich galaxy cluster, we will take into account the magnetic field in the galaxy cluster, attenuation of photons in extragalactic space and possible re-conversion in the galactic magnetic field. 

In [1]:
import sys
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rc('xtick', labelsize=20)   
plt.rc('ytick', labelsize=20)
plt.rcParams['axes.linewidth'] = 2.5
plt.rc('text', usetex=True)
plt.rc('font', family='serif',size=25)

In [2]:
import numpy as np
import astropy.units as u
from pathlib import Path
import glob
from astropy.io import fits
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion
import pickle

In [3]:
from scipy.stats import chi2
from functools import partial
import scipy.special as scipys

from gammaALPs.core import Source, ALP, ModuleList
from gammaALPs.base import environs, transfer

### Define a function to calculate the photon survival probability

In [4]:
def compute_ALP_absorption(modulelist, axion_mass, coupling, emin, emax, bins):
    ''' Input:
            -  modulelist:     ModuleList object assuming a given source
            -  axion_mass:     axion mass / 1 neV
            -  coupling  :     axion-gamma coupling / 1e-11 GeV^-1
            -  emin      :     min energy / GeV
            -  emin      :     max energy / GeV
            -  bins      :     number of points in energy log-sperated
        Output:
            -  energy points
            -  gamma absorption for the above energy points

    '''
    ebins            = np.logspace(np.log10(emin),np.log10(emax),bins)
    
    # unit conversion
    axion_mass       = axion_mass.to(u.neV)
    coupling         = coupling.to(1e-11/u.GeV)
    # passing m and g to the gammaALPs modulelist object
    modulelist.alp.m = axion_mass.value
    modulelist.alp.g = coupling.value
    modulelist.EGeV  = ebins

    px,  py,  pa     = modulelist.run(multiprocess=2)
    pgg              = px + py
    
    return modulelist.EGeV, pgg

In [12]:
source     = Source(z = 0.017559, ra = '03h19m48.1s', dec = '+41d30m42s') # this is for ngc1275

pin        = np.diag((1.,1.,0.)) * 0.5
alp        = ALP(0,0) 
modulelist = ModuleList(alp, source, pin = pin)
modulelist.add_propagation("ICMGaussTurb", 
          0, # position of module counted from the source. 
          nsim = 10, # number of random B-field realizations
          B0 = 25.,  # rms of B field
          n0 = 39.,  # normalization of electron density
          n2 = 4.05, # second normalization of electron density, see Churazov et al. 2003, Eq. 4
          r_abell = 500., # extension of the cluster
          r_core = 80.,   # electron density parameter, see Churazov et al. 2003, Eq. 4
          r_core2 = 280., # electron density parameter, see Churazov et al. 2003, Eq. 4
          beta = 1.2,  # electron density parameter, see Churazov et al. 2003, Eq. 4
          beta2= 0.58, # electron density parameter, see Churazov et al. 2003, Eq. 4
          eta = 0.5, # scaling of B-field with electron denstiy
          kL = 0.18, # maximum turbulence scale in kpc^-1, taken from A2199 cool-core cluster, see Vacca et al. 2012 
          kH = 9.,  # minimum turbulence scale, taken from A2199 cool-core cluster, see Vacca et al. 2012
          q = -2.80, # turbulence spectral index, taken from A2199 cool-core cluster, see Vacca et al. 2012
          seed=0 # random seed for reproducability, set to None for random seed.
         )
modulelist.add_propagation("EBL",1, model = 'dominguez') # EBL attenuation comes second, after beam has left cluster
modulelist.add_propagation("GMF",2, model = 'jansson12', model_sum = 'ASS') # finally, the beam enters the Milky Way Field

environs.py: 431 --- INFO: Using inputted chi
environs.py:1039 --- INFO: Using inputted chi


### Define the range of ALPs masses that we want to study 

#### Exercise: Define the numpy array of masses being 5 logarithmically scaled values from 1 neV to 1000 neV.

#### Solution

In [ ]:
%load ./Solutions/m_array.py

### Define the range of ALPs couplings to photons that we want to study

#### Exercise: Define the numpy array of couplings being 4 logarithmically scaled values from 8**e-10 GeV-1 to 5**e-10 GeV-1.

#### Solution

In [ ]:
%load ./Solutions/g_array.py

Let's compute the photon survival probability for the range of masses and couplings that we have just defined. 
Save the npy tables in "" 

In [17]:
%%time
j = 0
for m in m_array:
    for g in g_array:
        ms = m.to( 1e-2 * u.neV)
        gs = g.to( 1e-14 / u.GeV)

        enpoints, pgg   = compute_ALP_absorption(
                            modulelist = modulelist, # modulelist from gammaALP
                            axion_mass = m, # neV
                            coupling   = g , # 10^(-11) /GeV
                            emin       = 50,  # Gev
                            emax       = 2.2e4, # GeV
                            bins       = 100) # log-bins in enrgy for which computing the ALP-absorption

        arr = [enpoints] 
        for ipg in pgg:
            arr.append(ipg )
        arr = np.array(arr)
        file_name = f'ngc1275_100real_m_%0.f_g_%0.f_'  % (ms.value, gs.value) 
        np.save(  f"./Models/{file_name}.npy", arr) 
        

   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py: 644 --- INFO: Running Module 2: <class 'gammaALPs.base.environs.MixGMF'>
   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py: 644 --- INFO: Running Module 2: <class 'gammaALPs.base.environs.MixGMF'>
   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py: 644 --- INFO: Running Module 2: <class 'gammaALPs.base.environs.MixGMF'>
   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py: 644 --- INFO: Running Module 2: <class 'gammaALPs.base.environs.MixGMF'>
   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py: 644 --- INFO: Running Module 2: <class 'gammaALPs.base.environs.MixGMF'>
   core.py: 644 --- INFO: Running Module 0: <class 'gammaALPs.base.environs.MixICMGaussTurb'>
   core.py:

In [25]:
for m in m_array:
        for g in g_array:

            ms = m.to( 1e-2 * u.neV)
            gs = g.to( 1e-14 / u.GeV)


            fig, ax = plt.subplots(figsize=(15,10))
            ax.grid(True,which='both',linewidth=0.3)
            ax.set_ylabel('Photon survival probability', size=30)
            ax.set_xlabel('E [GeV]',size=30)
            ax.set_xlim([3e1,3e4])
            ax.set_ylim([0,1.05])
            ax.set_xscale("log")
            #ax.set_yscale("log")
            for ipg in pgg:
                #ax.scatter(enpoints, ipg,s=1)
                line,  =  ax.plot(enpoints, ipg)

            line.set_color("white")
            mlegend = m.to( 1 * u.neV)
            glegend = g.to( 1e-11 / u.GeV)

            str_leg = f'm \t = %0.f   {mlegend.unit}  \n g  \t = %0.1f   {glegend.unit}'  % (mlegend.value, glegend.value) 
            line.set_label(str_leg)
            ax.legend(fontsize = 25, loc="lower left", markerscale=1)


            file_name = f'%0.f_ngc1275_100real_m_%0.f_g_%0.f_'  % (j,ms.value, gs.value) 
            fig.savefig(f"./Models/png/{file_name}.png")
            fig.clf()
            #plt.show()

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

...

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

#### Exercise: Repeat the calculation of the photon survival probability, changing the central value of the strength of the galaxy cluster magnetic field to 15 muG and using only one magnetic field realisation. 

#### Solution

In [ ]:
%load ./Solutions/B_15_1real.py